In [ ]:
#import Dependecies: 
import requests 
from bs4 import BeautifulSoup
import pandas as pd

In [51]:
#Define the eBay filters dictionary 

url = "https://www.ebay.com/sch/i.html"
# Defie the query parameters for the search request 
params = {
    "_nkw": "iphone 15 pro max",
    "_sacat": 0,
    "_from": "R40",
    "LH_Sold": 1,
    "LH_Complete": 1,
    "rt": "nc",
    "LH_ItemCondition": "2010|2020|2030",
    "Network": "Unlocked",
    "_dcat": 9355,
     "_ipg": 240,
}



In [52]:
# Define headers to mimic a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}


In [53]:
#Create a prepared reqeust to inspect the full URL with parameters

request = requests.Request('GET', url, params=params)
prepared_request = request.prepare()
print(prepared_request.url)

https://www.ebay.com/sch/i.html?_nkw=iphone+15+pro+max&_sacat=0&_from=R40&LH_Sold=1&LH_Complete=1&rt=nc&LH_ItemCondition=2010%7C2020%7C2030&Network=Unlocked&_dcat=9355&_ipg=240


In [54]:
#initlized Variable 
page_number = 0 
items_list = []

#loop over page 
while True: 
    page_number += 1
    print(f"Scraping page {page_number}...")

    params['_pgn'] = page_number

    #Send get request to eBay with the defined parameters and headers: 
    response = requests.get(url, params=params, headers=headers)
    html_content = response.text # Get the HTML content of the page 

    #parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract items first
    items = soup.find_all('div', class_='s-item__wrapper clearfix')
    
    # Check if no items found - means we've reached the end
    if not items or len(items) <= 2:
        print('No more items found - reached the end')
        break
    
    # Extract Listings
    for item in items[2:]:
        title = item.find('div', class_='s-item__title').text
        price = item.find('span', class_='s-item__price').text
        link = item.find('a', class_='s-item__link')['href'].split('?')[0]
        image_url = item.find('div', class_='s-item__image-wrapper image-treatment').find('img').get('src','No image URL')

        # Define each item as a dictionary
        item_dict = {
            'Title': title,
            'Price': price,
            'Link': link,
            'Image Link': image_url
        }

        # Append the dictionary to the list
        items_list.append(item_dict)
    
    # Find the next button to check if there are more pages
    next_button = soup.find('button', class_='pagination__next', type='next')

    # Check if the next button is disabled or doesn't exist
    if not next_button or next_button.get('aria-disabled') == 'true':
        print('No more pages to scrape')
        break


Scraping page 1...
No more items found - reached the end


In [55]:
len(items_list)

0

In [56]:
# Debug: Let's check what we're actually getting from the page
response = requests.get(url, params=params, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Check for items with the specific class
items = soup.find_all('div', class_='s-item__wrapper clearfix')
print(f"Found {len(items)} items with class 's-item__wrapper clearfix'")

# Let's also try finding items with just 's-item__wrapper'
items_alt = soup.find_all('div', class_='s-item__wrapper')
print(f"Found {len(items_alt)} items with class 's-item__wrapper'")

# Let's see what classes are actually available
all_divs = soup.find_all('div', class_=lambda x: x and 's-item' in x)
print(f"\nFound {len(all_divs)} divs with 's-item' in class name")

# Print first few class names to see the pattern
if all_divs:
    print("\nFirst few div classes found:")
    for i, div in enumerate(all_divs[:5]):
        print(f"{i+1}. {div.get('class')}")


Found 0 items with class 's-item__wrapper clearfix'
Found 0 items with class 's-item__wrapper'

Found 0 divs with 's-item' in class name


In [57]:
# Let's look for the actual item containers
items_li = soup.find_all('li', class_=lambda x: x and 's-item' in str(x))
print(f"Found {len(items_li)} <li> elements with 's-item'")

if items_li:
    print("\nFirst item's classes:")
    print(items_li[0].get('class'))
    
    # Let's see the structure
    print("\nFirst item structure (first 500 chars):")
    print(str(items_li[0])[:500])


Found 0 <li> elements with 's-item'


In [58]:
# Let's try finding list items
all_li = soup.find_all('li')
print(f"Total <li> elements: {len(all_li)}")

# Check for items in unordered list
for li in all_li[:3]:
    classes = li.get('class', [])
    print(f"\nLI classes: {classes}")
    if classes:
        print(f"First 200 chars: {str(li)[:200]}")


Total <li> elements: 0


In [59]:
# Let's save a sample of the HTML to see what we're getting
with open('ebay_sample.html', 'w', encoding='utf-8') as f:
    f.write(response.text[:50000])  # First 50k chars
    
print("Sample HTML saved to ebay_sample.html")
print(f"Total HTML length: {len(response.text)} characters")

# Check if page contains certain keywords
if 'iPhone' in response.text:
    print("✓ Page contains 'iPhone'")
else:
    print("✗ Page does NOT contain 'iPhone'")
    
if 'Price' in response.text or 'price' in response.text:
    print("✓ Page contains price information")
else:
    print("✗ Page does NOT contain price information")


Sample HTML saved to ebay_sample.html
Total HTML length: 13229 characters
✗ Page does NOT contain 'iPhone'
✗ Page does NOT contain price information


In [60]:
# eBay likely embeds data in JSON within script tags
# Let's find them
import re
import json

# Find all script tags
scripts = soup.find_all('script', type=lambda x: x != 'text/javascript' or x is None)
print(f"Found {len(scripts)} script tags")

# Look for JSON data
for i, script in enumerate(scripts[:20]):  # Check first 20 scripts
    script_content = script.string
    if script_content and ('itemId' in script_content or 'listingId' in script_content or 'price' in script_content.lower()):
        print(f"\n=== Script {i} (first 500 chars) ===")
        print(script_content[:500])
        break


Found 10 script tags


In [61]:
# Let's try to find the srp-results ul
results_list = soup.find('ul', class_='srp-results')
if results_list:
    print("✓ Found srp-results list!")
    items = results_list.find_all('li', recursive=False)
    print(f"  Found {len(items)} direct child <li> elements")
    if items:
        print("\nFirst item classes:", items[0].get('class'))
        print("\nFirst item structure (first 800 chars):")
        print(str(items[0])[:800])
else:
    print("✗ Could not find srp-results list")
    # Try alternative
    print("\nTrying to find any ul with 'results' in class...")
    all_uls = soup.find_all('ul')
    for ul in all_uls:
        classes = ul.get('class', [])
        if classes and any('result' in str(c).lower() for c in classes):
            print(f"Found ul with classes: {classes}")


✗ Could not find srp-results list

Trying to find any ul with 'results' in class...


## Why items_list is empty (0 items)

**Problem:** eBay is blocking the scraper with a bot detection challenge page.

The HTML you're receiving says: "Checking your browser before you access eBay" - this is NOT the actual search results page.

### Solutions:

1. **Use Selenium with a real browser** - This can pass bot detection:
   ```python
   pip install selenium webdriver-manager
   ```

2. **Use eBay's Official API** - The proper way to get data:
   - [eBay Finding API](https://developer.ebay.com/DevZone/finding/Concepts/MakingACall.html)
   - [eBay Browse API](https://developer.ebay.com/api-docs/buy/browse/overview.html)
   - Requires registration but is reliable and legal

3. **Add delays and rotate User-Agents** - May help but not guaranteed:
   - Add `time.sleep(random.uniform(2, 5))` between requests
   - Rotate different user agents
   - Use proxy servers

4. **Use a scraping service** - Services like ScraperAPI, Bright Data handle anti-bot measures

**Recommended:** Use eBay's official API for reliable, legal access to their data.

## Selenium Implementation - Scraping with Real Browser

This approach uses Selenium with Chrome to bypass bot detection by simulating a real user browser.